# Chapter 14: The RTD and Size Distribution of Solids in Fluidized Beds

## Example 14.1: Flow_with_Elutriatio.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-14, Example 1, Page 343
//Title: Flow with Elutriation
//==========================================================================================================

clear
clc

//INPUT
Fo=2.7;//Feed rate in kg/min
Fof=0.9;//Feed rate of fines in feed in kg/min
Foc=1.8;//Feed rate of coarse in feed in kg/min
W=17;//Bed weight in kg
kf=0.8;//Elutriation of fines in min^-1
kc=0.0125;//Elutriation of coarse in min^-1

//CALCULATION
F1guess=1;//Guess value of F1
function[fn]=solver_func(F1)//Function defined for solving the system
    fn=F1-(Fof/(1+(W/F1)*kf))-(Foc/(1+(W/F1)*kc));//Eqn.(17)
endfunction
[F1]=fsolve(F1guess,solver_func,1E-6);//Inbuilt function fsolve to solve for F1
F1f=Fof/(1+(W/F1)*kf);//Flow rate of fines in entrained streams from Eqn.(16)
F1c=Foc/(1+(W/F1)*kc);//Flow rate of coarse in entrained streams from Eqn.(16)
F2f=Fof-F1f;//Flow rate of fines in overflow streams from Eqn.(9)
F2c=Foc-F1c;//Flow rate of coarse in overflow streams from Eqn.(9)
tbarf=1/((F1/W)+kf);//Mean residence time of fines from Eqn.(12)
tbarc=1/((F1/W)+kc);//Mean residence time of coarse from Eqn.(12)

//OUTPUT
mprintf('
Flow rate in entrained stream:
	Fines:%fkg/min
	Coarse:%fkg/min',F1f,F1c);
mprintf('
Flow rate in overflow stream:
	Fines:%fkg/min
	Coarse:%fkg/min',F2f,F2c);
mprintf('
Mean residence time:
	Fines:%fmins
	Coarse:%fmins',tbarf,tbarc);

//====================================END OF PROGRAM ======================================================

## Example 14.2: Flow_with_Elutriation_and_Change_in_Density_of_Solids.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-14, Example 2, Page 344
//Title: Flow with Elutriation and Change in Density of Solids
//==========================================================================================================

clear
clc

//INPUT
dt=4;//Diameter of reactor in m
ephsilonm=0.4;//Void fraction of static bed
rhos=2500;//Density of solid in the bed in kg/m^3
Lm=1.2;//Height of static bed in m
Fo=3000;//Feed rate in kg/hr
beta1=1.2;//Increase in density of solids
dp=[3;4;5;6;7;8;9;10;11;12;3;14;16;18;20;22;24;26;28;30]*10^-2;//Size of particles in mm
po=[0;0.3;0.8;1.3;1.9;2.6;3.5;4.4;5.7;6.7;7.5;7.8;7.5;6.3;5.0;3.6;2.4;1.3;0.5;0];//Size distribution of solids in mm^-1
k=[0;10;9.75;9.5;8.75;7.5;6.0;4.38;2.62;1.20;0.325;0;0;0;0;0;0;0;0;0]*10^-4;//Elutriation constant in s^-1
pi=3.14;

//CALCULATION
W=(pi/4*dt^2)*Lm*(1-ephsilonm)*rhos;//Weight of solids in bed
n=length(dp);
i=1;
F1guess=1000;//Guess value for F1
F1c=2510:10:2700;
while i<=n
    function[fn]=solver_func(F1)//Function defined for solving the system
        if k(i)==0  then    x(i)=0; break 
                    else    x(i)=(po(i)/(W*k(i)/F1))*log(1+(W*k(i)/F1));         
        end
    fn=F1/(Lm*Fo)-x(i);
    endfunction
    [F1(i)]=fsolve(F1guess,solver_func,1E-6);//Using inbuilt function fsolve for solving Eqn.(20) for F1
    c(i)=F1c(i)/(Lm*Fo);
    if F1(i)==0 then a(i)=0;
    else     a(i)=(po(i)/(W*k(i)/F1(i)))*log(1+(W*k(i)/F1(i)));
    end    
    i=i+1;
end
plot(F1,a,F1,c);
xtitle('F1 vs a,c','F1','a,c');
F1n=2500;//The point were both the curves meet
F2=beta1*Fo-F1n;//Flow rate of the second leaving stream
j=1;
m=length(dp);
while j<=m
    p1(j)=(1/F1n)*((Fo*po(j))/(1+(W/F1n)*k(j)));//Size distribution of stream 1 in mm^-1 from Eqn.(16)
    p2(j)=k(j)*W*p1(j)/F2;//Size distribution of stream 2 in mm^-1 from Eqn.(7)
    if p1(j)==0 & p2(j)==0 then tbar(j)=0;
    else  if  p1(j)==0 then tbar(j)=(W*p1(j))/(F2*p2(j));
        else if  p2(j)==0 then tbar(j)=(W*p1(j))/(F1n*p1(j));
            else tbar(j)=(W*p1(j))/(F1n*p1(j)+F2*p2(j));//Average time in hr from Eqn.(11)
            end
        end
    end
    j=j+1;
end

//OUTPUT
printf('
Flow rate of stream 1:%fkg/hr',F1n);
printf('
Flow rate of stream 2:%fkg/hr',F2);
j=1;
mprintf('
tbar(hr)');
while j<=m
    mprintf('
%f',tbar(j));
    j=j+1;
end

//====================================END OF PROGRAM ======================================================
//DISCLAIMER: The value obtained for tbar is deviating highly form the one given in textbook. However, the value obtained by manual calculation is close to the   ones obtained from the program.

## Example 14.3: Single_Size_Feed_of_Shrinking_Particles.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-14, Example 3, Page 351
//Title: Single-Size Feed of Shrinking Particles
//==========================================================================================================

clear
clc

//INPUT
dp=1;//Particle size in mm
Fo=10;//Feed rate in kg/min
k=0.1;//Particle shrinkage rate in mm/min

//CALCULATION
R=k/2;//Particle shrinkage rate in terms of radius
W=(Fo*dp/2)/(4*R);//Bed weight from Eqn.(42)

//OUTPUT
printf('
Weight of bed:%fkg',W);

//====================================END OF PROGRAM ======================================================

## Example 14.4: Wide_Size_Distribution_of_Shrinking_Particle.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-14, Example 4, Page 352
//Title: Wide Size Distribution of Shrinking Particle
//==========================================================================================================

clear
clc

//INPUT
dpi=[1.05;0.95;0.85;0.75;0.65;0.55;0.45;0.35;0.25;0.15;0.05];//Mean size in mm
Fo=[0;0.5;3.5;8.8;13.5;17.0;18.2;17.0;13.5;7.3;0]*10^-2;//Feed rate in kg/s
k=[0;0;0;0;0;0;0;0;2.0;12.5;62.5]*10^-5;//Elutriation constant in s^-1
R=-1.58*10^-5;//Rate of particle shrinkage in mm/s
deldpi=0.1;//Size intervals in mm

//CALCULATION
n=length(dpi);
m=2;//Starting with the largest value size interval that contains solids
W(m-1)=0;
while m<=n
    W(m)=(Fo(m)-R*W(m-1)/deldpi)/(k(m)-R/deldpi-3*R/dpi(m));//From Eqn.(33)
    m=m+1;
end
Wt=sum(W);//Total sum

//OUTPUT
printf('
Total mass in the bed:%fkg',Wt);

//====================================END OF PROGRAM ======================================================

## Example 14.5: Elutriation_and_Attrition_of_Catalyst.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-14, Example 5, Page 353
//Title: Elutriation and Attrition of Catalyst
//==========================================================================================================

clear
clc

//INPUT
dpi=[0.17;0.15;0.13;0.11;0.09;0.07;0.05;0.03;0.01];//Mean size of particles in mm
a=[0;0.95;2.45;5.2;10.1;23.2;35.65;20.0;2.45]*10^-2;//Feed composition Fo(dpi)/Fo
y=[0;0;0;0;0;0;0.625;10.225;159.25]*10^-6;//Elutriation and cyclone efficiency k(dpi)(1-eta(dpi))
F=0.01;//Rate at which solids are withdrawn in kg/s
W=40000;//Weight of bed in kg
dp1=0.11//Initial size in mm
dp2=0.085;//Size after shrinking in mm
dpmin=0.01;//Minimum size in mm
deldpi=2*10^-2;//Size inerval in mm
t=20.8;//Time in days
si=1;

//CALCULATION
kdash=log((dp1-dpmin)/(dp2-dpmin))/(t*24*3600);//Rate of particle shrinkage from Eqn.(24)
n=length(dpi);
m=2;
Fo=0.05;//Initial value of Fo
F1(m-1)=0;
s=0;
c=0;
t=1E-6;
while m<=n
    R(m)=-kdash*(dpi(m)-dpmin);//Rate of size change
    x(m)=(a(m)*Fo-W*R(m-1)*F1(m-1)/deldpi)/(F+(W*y(m))-(W*R(m)/deldpi)-3*W*R(m)/dpi(m));//Eqn.(34)
    F1(m)=x(m)*F;
    c=c+x(m);
    m=m+1;
    if abs(c-1)<t then break
    end
    Fo=Fo+0.0001;//Incrementing Fo
end  

//OUTPUT
mprintf('
Feed rate with deldpi=%fmm is %fg/hr',deldpi,Fo);
i=1;
mprintf('
Bed composition');
while i<=n
    printf('
%f',x(i)*100);
    i=i+1;
end

//====================================END OF PROGRAM ======================================================